In [49]:
import pandas as pd
import numpy as np

In [50]:
NUM_USERS = 1000
PROFESSOR_RATE = 0.7
STUDENT_RATE = 0.3

In [51]:
data = pd.read_csv('../scraping/results.csv')

In [52]:
data['student_count'] = data['student_count'] + 10
data['professor_count'] = data['professor_count'].clip(10)

In [53]:
data['student_rate'] = data['student_count'] / data['student_count'].sum()
data['professor_rate'] = data['professor_count'] / data['professor_count'].sum()

In [54]:
users = pd.DataFrame()

In [55]:
users['user_id'] = range(NUM_USERS)
users['is_professor'] = np.random.choice([False, True], p = [1 - PROFESSOR_RATE, PROFESSOR_RATE], size = NUM_USERS)
users['is_student'] = np.random.choice([False, True], p = [1 - STUDENT_RATE, STUDENT_RATE], size = NUM_USERS)

In [56]:
num_professors = users['is_professor'].sum()
num_students = users['is_student'].sum()

In [57]:
users.loc[users['is_professor'] == True, 'teaching_subject'] = np.random.choice(data['subject_eng'].values, num_professors, p = data['professor_rate'])

users.loc[users['is_student'] == True, 'learning_subject'] = np.random.choice(data['subject_eng'].values, num_students, p = data['student_rate'])


users = users[users['teaching_subject'] != users['learning_subject']]

In [58]:
users = users.dropna(subset = ['teaching_subject', 'learning_subject'], how = 'all')
users = users.drop(columns = ['is_professor', 'is_student'])

In [59]:
edges = pd.merge(users, users, left_on='teaching_subject', right_on='learning_subject', suffixes=('_teacher', '_student'))
edges = edges.dropna(subset=['teaching_subject_teacher', 'learning_subject_student'])
edges = edges[['user_id_teacher', 'user_id_student', 'teaching_subject_teacher']]
edges = edges.rename(columns = {'teaching_subject_teacher' : 'teaching'})

In [60]:
edges.to_csv('teacher_student_connections.csv', index = False)